## Code to search the contents for all of your SO answers

- File with
  - Tags
  - Title
  - ViewCount
  - Score
  - Answer ID
  - Accepted Answer Id
  - Answer Body

In [ ]:
import pandas as pd
from IPython.display import HTML
from bs4 import BeautifulSoup
from pathlib import Path
import re

In [ ]:
pd.set_option('display.max_columns', 700)
pd.set_option('display.max_rows', 400)
pd.set_option('display.min_rows', 10)
pd.set_option('display.expand_frame_repr', True)

- Perform query at [StackExchange Data Explorer][sede] with the following query, using your Stack Overflow user ID

```sql
SELECT 
    Ans.CreationDate, Qus.Tags, Qus.Title, Qus.ViewCount, Qus.Body AS QBody, Qus.Id AS QID,
    Ans.Score, Ans.ID, Qus.AcceptedAnswerId, Ans.Body
FROM Posts as Qus
LEFT JOIN Posts as Ans
ON Qus.ID = Ans.ParentID
WHERE Ans.OwnerUserId = ##UserId##
ORDER BY Ans.CreationDate DESCte DESC
```

- Save the file as `'so_QueryResults.csv'`

  [sede]: https://data.stackexchange.com/stackoverflow/query/new

In [ ]:
# load file
so = pd.read_csv('so_QueryResults.csv', dtype={'ID': int, 'AcceptedAnswerId': str})

In [ ]:
so.CreationDate = pd.to_datetime(so.CreationDate).dt.date

In [ ]:
so.fillna({'AcceptedAnswerId': 0}, inplace=True)  # replace NaN with 0
so.AcceptedAnswerId = so.AcceptedAnswerId.astype(int)  # convert from float to int
so['Accepted'] = so.ID == so.AcceptedAnswerId  # add a Boolean column True if your answer is the accepted answer

In [ ]:
# using the answer id, create a link to your answer
my_uid = 7758804
so['Answer Link'] = so[['ID', 'Title']].apply(lambda x: f'<a href="https://stackoverflow.com/a/{x['ID']}/{my_uid}">{x['Title']}</a>', axis=1)

In [ ]:
# question link
so['Question Link'] = so[['QID']].apply(lambda x: f'<a href="https://stackoverflow.com/q/{x['QID']}/{my_uid}">Question Link</a>', axis=1)

In [ ]:
# add Answer Link to the answer body as a header
so['Body'] = so[['Answer Link', 'Question Link', 'Body']].apply(lambda x: f'<h2>{x.iloc[0]}</h2>' + f'<h3>{x.iloc[1]}</h3>' + x.iloc[2], axis=1)

In [ ]:
so.drop(columns=['ID', 'AcceptedAnswerId', 'Title'], inplace=True)

In [ ]:
so.Tags = so.Tags.str.replace('><', ' ', regex=True).str.replace('<|>', '', regex=True).str.split()

# replace newline with html newline tag
so.QBody = so.QBody.str.replace("\\n+", "<br>", regex=True)
so.Body = so.Body.str.replace("\\n+", "<br>", regex=True)

In [ ]:
so.head()

In [ ]:
so.tail()

In [ ]:
so.Tags.explode(ignore_index=True).value_counts().head(20)

In [ ]:
# sort by ViewCount
# so = so.sort_values(['ViewCount'], ascending=False)  # ['Accepted', 'Score', 'ViewCount']  
# so.head(100)

In [ ]:
# sort by Accepted and CreationDate
# so = so.sort_values(['Accepted', 'CreationDate'], ascending=False).reset_index(drop=True)
# so.head(100)

In [ ]:
# various filters for search the Body of the answer
son = so[so.Body.str.lower().str.contains('bar_label', regex=False)]
# son = son[son.Body.str.lower().str.contains('year', regex=False)]
# son = son[son.Body.str.lower().str.contains('.plot', regex=False)]
# son = son[~son.Body.str.lower().str.contains('bar', regex=False)]
son

In [ ]:
so = so.drop(columns=['Body', 'QBody', 'Question Link', 'QID'])  # drop Body or the html will be a mess

In [ ]:
so.to_html('../trenton3983.github.io/files/so_answers_new.html', justify='center', index=False, escape=False)  # html to a file

In [ ]:
so.to_html('so_answers_body.html', columns=['QBody', 'Body'], justify='left', index=False, escape=False)

Header to add to html file. See [CSS Table Alignment][1] and [Try It][2]

```html
<!DOCTYPE html>
<html>
<head>
<!-- Google tag (gtag.js) -->
<script async src="https://www.googletagmanager.com/gtag/js?id=G-5XRBJ1N69L"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'G-5XRBJ1N69L');
</script>

<link rel="shortcut icon" href="../../../trenton_32px.jpg" type="image/x-icon" />/script>image/x-icon" />

<meta charset="utf-8" />
<meta name="keywords" content="stack overflow, numpy, dataframe, python, matplotlib, seaborn, pandas, json, visualization, data science, Portlandalytics Portland, Oregon">
<meta name="author" content="Trenton McKinney">
<meta name="description" content="Stack Overflow Answers by Trenton McKinney">

<meta name="viewport" content="width=device-width, initial-scale=1.0">

<title>Stack Overflow Answers by Trenton McKinney</title>

<script src="https://cdnjs.cloudflare.com/ajax/libs/require.js/2.1.10/require.min.js"></script>
<script src="https://cdnjs.cloudflare.com/ajax/libs/jquery/2.0.3/jquery.min.js"></script>
<style>
table {
  table-layout: fixed;
  width: 2400px;
}

td, th {
  width: 1200px;
  vertical-align: top;
  overflow: wrap;
}
</style>
</head>
</html>
```

 [1]: https://www.w3schools.com/css/css_table_align.asp
 [2]: https://www.w3schools.com/css/tryit.asp?filename=trycss_table_vertical-align

In [ ]:
HTML(so.head(1).to_html(escape=False, columns=['Body']).replace("\\n","<br>"))  # html displayed in notebook

## Print Title and Link

- For adding links into SO answer

In [ ]:
so['Link'] = so['ID'].apply(lambda x: f"https://stackoverflow.com/questions/{x}")

In [ ]:
so.head()

In [ ]:
so.iloc[0, -1]

In [ ]:
son = so[so.Body.str.lower().str.contains('bar_label', regex=False)]

In [ ]:
for i, v in enumerate(son.Title, 1):
    print(f'- [{v}][ex{i}]')
    
for i, v in enumerate(son.Link, 1):
    print(f'  [ex{i}]: {v}')

## Add head.html file to body html

In [ ]:
def insert_html_head(row):
    title, body, date = row

    # path to files directory
    files = Path(r'D:\users\trenton\Dropbox\PythonProjects\trenton3983.github.io\files')
    # path to so_answers directory
    so_answers = files / 'so_answers'
    # contents of head.html
    html = BeautifulSoup((files / 'head.html').read_text())

    head = html.find('head')
    body = BeautifulSoup(body).find('body')
    head.insert_after(body)

    # change the title
    _ = html.find('title').string.replace_with(title)

    # new file title
    file_title = str(date) + '_' + re.sub('[^A-Za-z0-9 ]+', '', title).replace(' ', '_').lower() + '.html'

    # write file
    (so_answers / file_title).open('w').write(str(html))

In [ ]:
_ = so[['Title', 'Body', 'CreationDate']].head(3).apply(insert_html_head, axis=1)